# Sarmalayıcı Yöntem (Wrapper Method) - RFE + Lojistik Regresyon

Bu notebook, **Recursive Feature Elimination (RFE)** yöntemini **Lojistik Regresyon** algoritması ile birlikte kullanarak özellik seçimi yapar.

## Yöntem Açıklaması
- RFE, özellikleri iteratif olarak eler
- Her iterasyonda en düşük öneme sahip özellik(ler) çıkarılır
- Lojistik Regresyon modeli temel tahmin edici olarak kullanılır
- En son kalan 15 özellik seçilir

**Referans:** https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html

In [ ]:
# Gerekli kütüphaneleri import et
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Helper modüllerini import et
from dataset_helper import (
    load_processed_dataset,
    split_features_target,
    save_selected_dataset,
    normalize_scores
)
from file_helper import write_report
from report_helper import generate_analysis_report

## 1. Veri Kümesini Yükle

In [ ]:
# İşlenmiş veri kümesini yükle
df = load_processed_dataset()

# Özellik ve hedef değişken olarak ayır
X, y = split_features_target(df, target_column='is_popular')

print(f"\nÖzellik isimleri:")
print(X.columns.tolist())

## 2. Veriyi Ölçeklendir

Lojistik Regresyon için verileri standardize etmek önemlidir.

In [ ]:
# Verileri standardize et
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# DataFrame olarak sakla (sütun isimleri için)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

print(f"Ölçeklendirilmiş veri boyutu: {X_scaled_df.shape}")

## 3. RFE ile Özellik Seçimi

In [ ]:
# Lojistik Regresyon modeli oluştur
# max_iter artırıldı çünkü yakınsama sorunları olabilir
estimator = LogisticRegression(
    max_iter=1000,
    solver='lbfgs',
    random_state=42,
    n_jobs=-1
)

# RFE ile 15 özellik seç
TOP_N = 15

print(f"RFE ile {TOP_N} özellik seçiliyor...")
print("Bu işlem birkaç dakika sürebilir...\n")

rfe = RFE(
    estimator=estimator,
    n_features_to_select=TOP_N,
    step=1,  # Her adımda 1 özellik ele
    verbose=1
)

# RFE uygula
rfe.fit(X_scaled, y)

print("\nRFE tamamlandı!")

## 4. Seçilen Özellikleri ve Sıralamayı Göster

In [ ]:
# RFE sonuçlarını al
feature_names = X.columns.tolist()

# Seçilen özellikler (support_) ve sıralama (ranking_)
# ranking_ değeri düşük olan daha önemli (1 = seçildi)
selected_mask = rfe.support_
feature_ranking = rfe.ranking_

print("=== RFE SONUÇLARI ===")
print(f"Seçilen özellik sayısı: {sum(selected_mask)}")
print(f"\nÖzellik sıralaması (1 = en önemli/seçildi):")

# DataFrame oluştur
rfe_results_df = pd.DataFrame({
    'Özellik': feature_names,
    'RFE Sırası': feature_ranking,
    'Seçildi': selected_mask
})

# Sıraya göre sırala
rfe_results_df = rfe_results_df.sort_values('RFE Sırası')
rfe_results_df = rfe_results_df.reset_index(drop=True)

print(rfe_results_df.to_string(index=False))

In [ ]:
# Seçilen özellikleri listele
selected_features = [feature_names[i] for i in range(len(feature_names)) if selected_mask[i]]

print(f"\n=== SEÇİLEN EN İYİ {TOP_N} ÖZELLİK ===")
for i, feature in enumerate(selected_features, 1):
    print(f"{i:2d}. {feature}")

## 5. Normalize Skorlar Oluştur

In [ ]:
# RFE sıralamasını normalize skora dönüştür
# Düşük sıra = daha önemli, bu yüzden ters çeviriyoruz
max_rank = max(feature_ranking)
importance_scores = max_rank - feature_ranking + 1  # Ters çevir
normalized_scores = normalize_scores(importance_scores)

# Sıralama DataFrame'i oluştur
ranking_df = pd.DataFrame({
    'Özellik': feature_names,
    'RFE Sırası': feature_ranking,
    'Ham Skor': importance_scores,
    'Mutlak Skor': importance_scores,  # RFE için ham = mutlak
    'Normalize Skor': normalized_scores
})

# Önem sırasına göre sırala
ranking_df = ranking_df.sort_values('RFE Sırası')
ranking_df = ranking_df.reset_index(drop=True)
ranking_df['Sıra'] = range(1, len(ranking_df) + 1)

# Sütun sırasını düzenle
ranking_df = ranking_df[['Sıra', 'Özellik', 'RFE Sırası', 'Ham Skor', 'Mutlak Skor', 'Normalize Skor']]

print("\n=== TÜM ÖZELLİKLERİN SIRALAMASI ===")
print(ranking_df.to_string(index=False))

## 6. Görselleştirme

In [ ]:
import matplotlib.pyplot as plt

# Seçilen özellikler için bar plot
top_15_df = ranking_df.head(TOP_N).copy()

plt.figure(figsize=(12, 8))
colors = ['#2ecc71' if idx < 15 else '#3498db' for idx in range(len(top_15_df))]
plt.barh(range(len(top_15_df)), top_15_df['Normalize Skor'], color=colors)
plt.yticks(range(len(top_15_df)), top_15_df['Özellik'])
plt.xlabel('Normalize Önem Skoru')
plt.title('En İyi 15 Özellik - RFE ile Lojistik Regresyon')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.savefig('photos/wrapper_method_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print("Grafik kaydedildi: wrapper_method_importance.png")

## 7. Analiz Raporu Oluştur

In [ ]:
# Rapor için yöntem açıklaması
method_description = """
Sarmalayıcı yöntem olarak **Recursive Feature Elimination (RFE)** kullanılmıştır. 
Bu yöntem, **Lojistik Regresyon** algoritması ile birlikte uygulanmıştır.

### Yöntem Detayları:
- RFE, başlangıçta tüm özelliklerle başlar
- Her iterasyonda model eğitilir ve en az önemli özellik(ler) elenir
- Bu işlem istenilen özellik sayısına ulaşılana kadar devam eder
- Lojistik Regresyon katsayıları özellik önemini belirler

### Model Parametreleri:
- **Estimator:** LogisticRegression
- **Solver:** lbfgs
- **Max Iterations:** 1000
- **Step:** 1 (her adımda 1 özellik ele)

### Avantajları:
- Model performansını doğrudan optimize eder
- Özellikler arası etkileşimleri dikkate alır
- Sıralı özellik seçimi sağlar

### Dezavantajları:
- Hesaplama maliyeti yüksek olabilir
- Seçilen modele bağımlı
"""

# Dataset bilgisi
dataset_info = {
    "name": "processed_dataset.csv",
    "shape": f"{df.shape}",
    "source": "UCI Online News Popularity"
}

# Ek notlar
additional_notes = """
RFE yöntemi, Lojistik Regresyon modelinin katsayılarını kullanarak özellik önemini belirler.
Bu nedenle, seçilen özellikler Lojistik Regresyon için en ayırt edici olanlardır.

**Not:** Veriler StandardScaler ile ölçeklendirilmiştir, bu Lojistik Regresyon için önemlidir.
"""

# Rapor oluştur
report_content = generate_analysis_report(
    method_name="Sarmalayıcı Yöntem - RFE + Lojistik Regresyon",
    method_description=method_description,
    ranking_df=ranking_df,
    top_n=TOP_N,
    dataset_info=dataset_info,
    additional_notes=additional_notes
)

# Rapor kaydet
write_report(report_content, "wrapper_analysis_report.md")

print("\nRapor başarıyla oluşturuldu!")

## 8. Seçilen Özelliklerle Veri Kümesi Oluştur

In [ ]:
# Seçilen özelliklerle yeni veri kümesi oluştur ve kaydet
output_file = "wrapper_method_selected_dataset.csv"

save_selected_dataset(
    df=df,
    selected_features=selected_features,
    target_column='is_popular',
    filename=output_file
)

print(f"\n✅ Sarmalayıcı yöntemi tamamlandı!")
print(f"   - {TOP_N} özellik seçildi")
print(f"   - Veri kümesi: {output_file}")
print(f"   - Rapor: wrapper_analysis_report.md")

## 9. Özet

In [ ]:
print("="*60)
print("SARMALAYICI YÖNTEMİ - ÖZET")
print("="*60)
print(f"Kullanılan Yöntem: RFE + Lojistik Regresyon")
print(f"Toplam Özellik Sayısı: {len(feature_names)}")
print(f"Seçilen Özellik Sayısı: {TOP_N}")
print(f"\nSeçilen Özellikler:")
for i, feat in enumerate(selected_features, 1):
    print(f"  {i}. {feat}")
print("="*60)